In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import csv
import time
import os

In [22]:
options = Options()
#options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)

#main forum page
driver.get("https://www.watchuseek.com/forums/reviews.67/")
time.sleep(5) 

#thread links
threads = driver.find_elements(By.CSS_SELECTOR, "h3.structItem-title > a")
thread_urls = [t.get_attribute("href") for t in threads if "/threads/" in t.get_attribute("href")]

#showing that links were grabbed successfully
for t in threads:
    print(t.text, t.get_attribute("href"))

driver.quit()


Sticky https://www.watchuseek.com/threads/the-review-forum-is-for-reviews-only.90159/
The Review Forum is for reviews only>>>> https://www.watchuseek.com/threads/the-review-forum-is-for-reviews-only.90159/
A review of a new Oceaneva Deep Marine Explorer VI https://www.watchuseek.com/threads/a-review-of-a-new-oceaneva-deep-marine-explorer-vi.5635925/
Color, Character, and Value: The Farer Lissom Forrest Reviewed - Joyful Design & Thoughtful Details https://www.watchuseek.com/threads/color-character-and-value-the-farer-lissom-forrest-reviewed-joyful-design-thoughtful-details.5635505/
Chrono24 Review https://www.watchuseek.com/threads/chrono24-review.5557515/
Franck Dubarry Crazy Wheels watch review https://www.watchuseek.com/threads/franck-dubarry-crazy-wheels-watch-review.5527287/
What Watches Jewelers Wear on 47th https://www.watchuseek.com/threads/what-watches-jewelers-wear-on-47th.5635332/
Henry Archer Verden GMT https://www.watchuseek.com/threads/henry-archer-verden-gmt.5634546/
Add

In [3]:
options = Options()
#options.add_argument("--headless=new") 
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)

#main forum page
driver.get("https://www.watchuseek.com/forums/reviews.67/")
time.sleep(5)

thread_data = []

#thread links
thread_items = driver.find_elements(By.CSS_SELECTOR, "div.structItem--thread")
thread_links = []

for item in thread_items:
    try:
        title_elem = item.find_element(By.CSS_SELECTOR, "h3.structItem-title > a")
        title = title_elem.text.strip()
        url = title_elem.get_attribute("href")

        thread_links.append((title, url))
    except Exception as e:
        print(f"Failed to extract thread link: {e}")

print(f"{len(thread_links)} threads")

#main post (review)
for title, url in thread_links:
    try:
        driver.get(url)

        #javascript load time
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "article[data-post-id]"))
        )

        post_elem = driver.find_element(By.CSS_SELECTOR, "article[data-post-id]")
        post_text = post_elem.text.strip()

        thread_data.append({
            "title": title,
            "url": url,
            "review": post_text
        })

        print(f"Scraped: {title}")

    except Exception as e:
        print(f"Failed on {url}: {e}")
        continue

#csv
csv_filename = "watchuseek_reviews.csv"
with open(csv_filename, "w", encoding="utf-8", newline='') as f:
    writer = csv.DictWriter(f, fieldnames=["title", "url", "review"])
    writer.writeheader()
    writer.writerows(thread_data)

driver.quit()
print(f"\n {len(thread_data)} reviews saved to {csv_filename}") #see number of successful reviews scrapped


36 threads
Failed on https://www.watchuseek.com/threads/the-review-forum-is-for-reviews-only.90159/: Message: 
Stacktrace:
0   chromedriver                        0x00000001007b355c cxxbridge1$str$ptr + 2731064
1   chromedriver                        0x00000001007ab454 cxxbridge1$str$ptr + 2698032
2   chromedriver                        0x00000001002fa3f8 cxxbridge1$string$len + 90664
3   chromedriver                        0x000000010034171c cxxbridge1$string$len + 382284
4   chromedriver                        0x0000000100382b1c cxxbridge1$string$len + 649548
5   chromedriver                        0x0000000100335a0c cxxbridge1$string$len + 333884
6   chromedriver                        0x00000001007765e0 cxxbridge1$str$ptr + 2481340
7   chromedriver                        0x0000000100779848 cxxbridge1$str$ptr + 2494244
8   chromedriver                        0x0000000100757234 cxxbridge1$str$ptr + 2353424
9   chromedriver                        0x000000010077a104 cxxbridge1$str$ptr 

* Able to find 36 threads (all threads on first page)
* 0 reviews extracted
* "article[data-post-id]" as selector not working
* watching the chromedriver, the main post (review) is seen on screen

In [7]:
options = Options()
#options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)

#main forum page
driver.get("https://www.watchuseek.com/forums/reviews.67/")
time.sleep(5)

thread_data = []

#thread links
thread_links = []
thread_items = driver.find_elements(By.CSS_SELECTOR, "div.structItem--thread")

for item in thread_items:
    try:
        title_elem = item.find_element(By.CSS_SELECTOR, "h3.structItem-title > a")
        title = title_elem.text.strip()
        url = title_elem.get_attribute("href")

        thread_links.append((title, url))
    except Exception as e:
        print(f"Failed to extract thread link: {e}")

print(f"{len(thread_links)} threads")

#main post (review)
for title, link in thread_links:
    try:
        driver.get(link)
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "article[data-post-id] div.bbWrapper")) #waiting for post body text to load
        )
        time.sleep(2)

        content = driver.find_element(By.CSS_SELECTOR, "article[data-post-id] div.bbWrapper").text.strip() #div.bbWrapper (class for body text of post)
        thread_data.append({
            "title": title,
            "url": link,
            "review": content
        })

        print(f"Scraped: {title}")

    except Exception as e:
        print(f"Error scraping {link}: {e}")

#csv
csv_filename = "watchuseek_reviews.csv"
with open(csv_filename, "w", encoding="utf-8", newline='') as f:
    writer = csv.DictWriter(f, fieldnames=["title", "url", "review"])
    writer.writeheader()
    writer.writerows(thread_data)

driver.quit()
print(f"\n {len(thread_data)} reviews saved to {csv_filename}") #see number of successful reviews scrapped


36 threads
Error scraping https://www.watchuseek.com/threads/the-review-forum-is-for-reviews-only.90159/: Message: 
Stacktrace:
0   chromedriver                        0x000000010330755c cxxbridge1$str$ptr + 2731064
1   chromedriver                        0x00000001032ff454 cxxbridge1$str$ptr + 2698032
2   chromedriver                        0x0000000102e4e3f8 cxxbridge1$string$len + 90664
3   chromedriver                        0x0000000102e9571c cxxbridge1$string$len + 382284
4   chromedriver                        0x0000000102ed6b1c cxxbridge1$string$len + 649548
5   chromedriver                        0x0000000102e89a0c cxxbridge1$string$len + 333884
6   chromedriver                        0x00000001032ca5e0 cxxbridge1$str$ptr + 2481340
7   chromedriver                        0x00000001032cd848 cxxbridge1$str$ptr + 2494244
8   chromedriver                        0x00000001032ab234 cxxbridge1$str$ptr + 2353424
9   chromedriver                        0x00000001032ce104 cxxbridge1$str

* Still successfully found the threads
* 0 reviews extracted despite change in selector

In [8]:
options = Options()
# ptions.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)

#taking screenshot of thread pages
os.makedirs("debug_screens", exist_ok=True)

#main forum page
driver.get("https://www.watchuseek.com/forums/reviews.67/")
time.sleep(5)

thread_data = []

#thread links
thread_items = driver.find_elements(By.CSS_SELECTOR, "div.structItem--thread")
thread_links = []

for item in thread_items:
    try:
        title_elem = item.find_element(By.CSS_SELECTOR, "h3.structItem-title > a")
        title = title_elem.text.strip()
        url = title_elem.get_attribute("href")

        thread_links.append((title, url))
    except Exception as e:
        print(f"Failed to extract thread link: {e}")

print(f"{len(thread_links)} threads")

#main post (review)
for i,(title, link) in enumerate(thread_links):
    try:
        driver.get(link)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "article[data-post-id]"))
        )
        time.sleep(2)

        #get all posts in thread
        posts = driver.find_elements(By.CSS_SELECTOR, "article[data-post-id]")
        if not posts:
            raise Exception("No posts found")

        first_post = posts[0]
        body = first_post.find_element(By.CSS_SELECTOR, "div.bbWrapper")
        print("Found posts:", len(posts))
        print("Raw HTML preview (first 200 chars):")
        print(body.get_attribute("innerHTML")[:200])

        review = body.text.strip()

        print("Extracted review (preview):", review[:200])

        thread_data.append({
            "title": title,
            "url": link,
            "review": review
        })

        print(f"Scraped: {title}")

    except Exception as e:
        print(f"Error scraping {link}: {type(e).__name__}: {e}")
        driver.save_screenshot(f"debug_screens/fail_{i+1}.png") #saves screenshot for particular thread with number for identification


#csv
csv_filename = "watchuseek_reviews.csv"
with open(csv_filename, "w", encoding="utf-8", newline='') as f:
    writer = csv.DictWriter(f, fieldnames=["title", "url", "review"])
    writer.writeheader()
    writer.writerows(thread_data)

driver.quit()
print(f"\n {len(thread_data)} reviews saved to {csv_filename}") #see number of successful reviews scrapped


36 threads
Error scraping https://www.watchuseek.com/threads/the-review-forum-is-for-reviews-only.90159/: TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x000000010064355c cxxbridge1$str$ptr + 2731064
1   chromedriver                        0x000000010063b454 cxxbridge1$str$ptr + 2698032
2   chromedriver                        0x000000010018a3f8 cxxbridge1$string$len + 90664
3   chromedriver                        0x00000001001d171c cxxbridge1$string$len + 382284
4   chromedriver                        0x0000000100212b1c cxxbridge1$string$len + 649548
5   chromedriver                        0x00000001001c5a0c cxxbridge1$string$len + 333884
6   chromedriver                        0x00000001006065e0 cxxbridge1$str$ptr + 2481340
7   chromedriver                        0x0000000100609848 cxxbridge1$str$ptr + 2494244
8   chromedriver                        0x00000001005e7234 cxxbridge1$str$ptr + 2353424
9   chromedriver                        0x000000010060a

* still failing
* screenshots showing that at least visually, the posts (reviews) are loaded and should be able to be extracted

In [10]:
#testing a single thread
driver.get("https://www.watchuseek.com/threads/a-review-of-a-new-oceaneva-deep-marine-explorer-vi.5635925/")
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "article[data-post-id]"))
)
time.sleep(3)

posts = driver.find_elements(By.CSS_SELECTOR, "article[data-post-id]")
print("Posts found:", len(posts))

first_post = posts[0]
body = first_post.find_element(By.CSS_SELECTOR, "div.bbWrapper")
review = body.text.strip()
print("Review:", review[:500])


MaxRetryError: HTTPConnectionPool(host='localhost', port=56659): Max retries exceeded with url: /session/512d88ff65d4ecde56049597639b69e7/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1037117c0>: Failed to establish a new connection: [Errno 61] Connection refused'))